In [2]:
% matplotlib inline

UsageError: Line magic function `%` not found.


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd
from rdkit import Chem
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops
from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool
from torch_geometric.data import Data

In [4]:
num_atom_type = 180 # including the extra mask tokens
num_chirality_tag = 10

num_bond_type = 5 # including aromatic and self-loop edge
num_bond_direction = 3 

class GINEConv(MessagePassing):
    def __init__(self, emb_dim):
        super(GINEConv, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(emb_dim, 2*emb_dim), 
            nn.ReLU(), 
            nn.Linear(2*emb_dim, emb_dim)
        )
        self.edge_embedding1 = nn.Embedding(num_bond_type, emb_dim)
        self.edge_embedding2 = nn.Embedding(num_bond_direction, emb_dim)
        nn.init.xavier_uniform_(self.edge_embedding1.weight.data)
        nn.init.xavier_uniform_(self.edge_embedding2.weight.data)

    def forward(self, x, edge_index, edge_attr):
        # add self loops in the edge space
        edge_index = add_self_loops(edge_index, num_nodes=x.size(0))[0]

        # add features corresponding to self-loop edges.
        self_loop_attr = torch.zeros((x.size(0), edge_attr.size(1)), dtype=torch.long)
        self_loop_attr[:,0] = 4 # bond type for self-loop edge
        self_loop_attr = self_loop_attr.to(edge_attr.device).to(edge_attr.dtype)
        edge_attr = torch.cat((edge_attr, self_loop_attr), dim=0)

        # 确保边属性索引为整数类型
        edge_attr_indices = edge_attr.long()

        edge_embeddings = self.edge_embedding1(edge_attr_indices[:, 0]) + \
                          self.edge_embedding2(edge_attr_indices[:, 1])

        return self.propagate(edge_index, x=x, edge_attr=edge_embeddings)

    def message(self, x_j, edge_attr):
        return x_j + edge_attr

    def update(self, aggr_out):
        return self.mlp(aggr_out)


class GINet(nn.Module):
    """
    Args:
        num_layer (int): the number of GNN layers
        emb_dim (int): dimensionality of embeddings
        max_pool_layer (int): the layer from which we use max pool rather than add pool for neighbor aggregation
        drop_ratio (float): dropout rate
        gnn_type: gin, gcn, graphsage, gat
    Output:
        node representations
    """
    def __init__(self, num_layer=5, emb_dim=300, feat_dim=256, drop_ratio=0, pool='mean',num_atom_features=9):
        super(GINet, self).__init__()
        self.num_layer = num_layer
        self.emb_dim = emb_dim
        self.feat_dim = feat_dim
        self.drop_ratio = drop_ratio

        self.x_embedding1 = nn.Embedding(num_atom_type, emb_dim)
        self.x_embedding2 = nn.Embedding(num_chirality_tag, emb_dim)
        nn.init.xavier_uniform_(self.x_embedding1.weight.data)
        nn.init.xavier_uniform_(self.x_embedding2.weight.data)
        self.atom_feature_lin = nn.Linear(num_atom_features, emb_dim)  

        # List of MLPs
        self.gnns = nn.ModuleList()
        for layer in range(num_layer):
            self.gnns.append(GINEConv(emb_dim))

        # List of batchnorms
        self.batch_norms = nn.ModuleList()
        for layer in range(num_layer):
            self.batch_norms.append(nn.BatchNorm1d(emb_dim))
        
        if pool == 'mean':
            self.pool = global_mean_pool
        elif pool == 'max':
            self.pool = global_max_pool
        elif pool == 'add':
            self.pool = global_add_pool
        
        self.feat_lin = nn.Linear(self.emb_dim, self.feat_dim)

        self.out_lin = nn.Sequential(
            nn.Linear(self.feat_dim, self.feat_dim), 
            nn.ReLU(inplace=True),
            nn.Linear(self.feat_dim, self.feat_dim//2)
        )

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        edge_attr = data.edge_attr
        h = self.atom_feature_lin(x)
        # h = self.x_embedding1(x[:,0]) + self.x_embedding2(x[:,1])

        for layer in range(self.num_layer):
            h = self.gnns[layer](h, edge_index, edge_attr)
            h = self.batch_norms[layer](h)
            if layer == self.num_layer - 1:
                h = F.dropout(h, self.drop_ratio, training=self.training)
            else:
                h = F.dropout(F.relu(h), self.drop_ratio, training=self.training)

        h = self.pool(h, data.batch)
        h = self.feat_lin(h)
        out = self.out_lin(h)
        
        return h, out


In [5]:
def atom_features(atom):
    """返回单个原子的特征向量"""
    atomic_num = atom.GetAtomicNum()
    
    return [
        atomic_num,  # 原子序数
        atom.GetDegree(),  # 原子的度
        atom.GetExplicitValence(),  # 显式价态
        atom.GetFormalCharge(),  # 形式电荷
        atom.GetNumRadicalElectrons(),  # 自由基电子数
        # electronegativity.get(atomic_num, 0),  # 电负性
        atom.GetHybridization().real,  # 杂化类型
        1 if atom.GetChiralTag() != Chem.ChiralType.CHI_UNSPECIFIED else 0,  # Chirality
        1 if atom.IsInRing() else 0,  # 原子是否在环中
        1 if atom.GetIsAromatic() else 0,  # 是否为芳香原子

        # ... 添加其他您需要的原子特征
    ]

def bond_features(bond):
    """返回单个键的特征向量"""
    bt = bond.GetBondType()
    return [
        1 if bt == Chem.rdchem.BondType.SINGLE else 0,
        1 if bt == Chem.rdchem.BondType.DOUBLE else 0,
        1 if bt == Chem.rdchem.BondType.TRIPLE else 0,
        1 if bt == Chem.rdchem.BondType.AROMATIC else 0,
        # ... 添加其他您需要的键特征
    ]

In [6]:
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    # 创建节点和边特征
    # 计算每个原子的特征向量
    atom_features_list = [atom_features(atom) for atom in mol.GetAtoms()]
    x = torch.tensor(atom_features_list, dtype=torch.float)
    edge_indices = []
    edge_attrs = []
    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        edge_indices.append((i, j))
        edge_indices.append((j, i))  # 反向边
        edge_attrs += [bond_features(bond), bond_features(bond)]
    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attrs, dtype=torch.long)
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)


In [7]:
from rdkit import Chem
from rdkit.Chem import rdchem
import numpy as np
electronegativity = {
    1: 2.20,  # H
    6: 2.55,  # C
    7: 3.04,  # N
    8: 3.44,  # O
    9: 3.98,  # F
    15: 2.19, # P
    16: 2.58, # S
    17: 3.16, # Cl
    35: 2.96, # Br
    53: 2.66, # I
    34: 2.55, # Se
    # ... other elements ...
}

def extract_heteroatom_electronegativity(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return []

    heteroatoms_electronegativity = []
    for atom in mol.GetAtoms():
        atomic_num = atom.GetAtomicNum()
        # 如果原子不是氢（原子序数1）或碳（原子序数6），则认为是杂原子
        if atomic_num not in [1, 6]:
            electroneg = electronegativity.get(atomic_num, 0)
            heteroatoms_electronegativity.append(electroneg)

    return heteroatoms_electronegativity

# 加载数据
df = pd.read_csv('data/NFAs_cleaned.csv')

# 初始化模型
model = GINet(num_layer=3, emb_dim=64, feat_dim=64, num_atom_features=9)

# 提取特征
features = []
for idx, row in df.iterrows():
    smiles = row['SMILES']
    data = smiles_to_graph(smiles)
    electronegativities = extract_heteroatom_electronegativity(smiles)

    if data is not None:
        feature, _ = model(data)
        feature_np = feature.detach().numpy()

        # 将电负性数据转换为二维数组
        electronegativities_np = np.array([electronegativities])

        # 将电负性数据加到特征后面
        combined_feature = np.concatenate([feature_np, electronegativities_np], axis=1)
        features.append(combined_feature)
    else:
        features.append(None)

# 将特征添加到DataFrame
df['Features'] = features

# 保存处理后的DataFrame
df.to_csv('data/NFAs_GINE_features.csv', index=False)